# MAIS 202 Assignment 4: Fashion MNIST challenge

By Anthony Dumais and Alberto Lopez-Garcia

We were inspired by the organization of the [Fashion_MNIST_TensorFlow_classification](https://www.kaggle.com/juniorbueno/fashion-mnist-tensorflow-classification) notebook and implemented 2D convolutional neural networks with the valuable help of the tutorial [Deep Learning from Scratch](https://www.kaggle.com/dansbecker/exercise-deep-learning-from-scratch-daily).

## 1) Preprocessing
### Importing libraries and making sure our datasets are in the correct directory

We will work with the datasets provided in the Kaggle competition.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import csv

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Loading and visualizing data
Load the train values and labels into X_train and y_train respectively.

In [ ]:
img_rows, img_cols = 28, 28
num_classes = 10

X_train = np.load("/kaggle/input/fashionmnistchallenge/train_images.npy/train_images.npy")

with open("/kaggle/input/fashionmnistchallenge/train_labels.csv") as csv_file:
    csv_reader = csv.reader(csv_file)
    colnames = next(csv_reader)  # skip column names
    y_train = np.array(list(csv_reader)).astype(float)
    y_train = np.delete(y_train, 0, axis=1) # delete index column

We display the first elements of X_train and y_train to make sure we have the correct data format.

In [ ]:
X_train[0][0], y_train[0]

Visualize an image using the given code block.

In [ ]:
def show_image(arr):
    two_d = (np.reshape(arr, (img_rows, img_cols)) * 255).astype(np.uint8)
    plt.imshow(two_d, interpolation='nearest')
    plt.show()

show_image(X_train[0]) # 0 is the index of the training image you want to display

### Normalizing images and reshaping database

In [ ]:
X_train = X_train / 255.0

# As the dimension of each image is 28x28, we changed the entire database to the format [-1 (all elements), height * width]
X_train = X_train.reshape(-1, img_rows*img_cols)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

## 2) Building the Artificial Neural Network

### Defining the model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D

fashion_model = tf.keras.models.Sequential()

### Adding the first dense (fully-connected) layer

In [ ]:
fashion_model.add(Dense(units=128,
                        activation='relu',
                        input_shape = (img_rows*img_cols, )))

### Adding the rest of the layers

In [ ]:
fashion_model.add(Dense(units=128,
                        activation='relu'))
fashion_model.add(Dense(units=128,
                        activation='relu'))
fashion_model.add(Flatten())

### Adding the output layer

In [ ]:
# Prediction layer. It has 10 units, each corresponding to a different class.
fashion_model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

### Compiling the model

In [ ]:
fashion_model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['sparse_categorical_accuracy'])

In [ ]:
fashion_model.summary()

## 3) Fitting the model

In [ ]:
fashion_model.fit(X_train, y_train, epochs=5)

## 4) Making predictions and creating output file

### Preprocessing test data

In [ ]:
X_test = np.load("/kaggle/input/fashionmnistchallenge/test_images.npy/test_images.npy")
X_test = X_test / 255.0
X_test = X_test.reshape(-1, img_rows*img_cols)

### Generating predictions

In [ ]:
y_test = [np.argmax(i) for i in fashion_model.predict(X_test)]

### Creating CSV file from prediction list

In [ ]:
df_test = pd.read_csv('../input/fashionmnistchallenge/sample_submission.csv')
df_test['label'] = y_test
df_test.to_csv('submission.csv', index=False)